In [1]:

!apt install gdal-bin python-gdal python3-gdal 
!apt install python3-rtree 
!pip install git+git://github.com/geopandas/geopandas.git
!pip install descartes 
!pip install folium 
!pip install plotly_express
!pip install -U kaleido

import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import folium

import plotly 
import plotly.offline as py
import plotly.graph_objs as go
import plotly_express as px

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import io

df = pd.read_csv('Furniture.csv')

df=df.drop(['is_open','attributes','categories'], axis=1)
df['hours'] = df['hours'].fillna(0)
df['postal_code'] = df['postal_code'].fillna(0)
df = df.drop(['address'],axis=1)
df=df.dropna()

best_shops = df.sort_values(by=['review_count','stars'], ascending=False)[:20]
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")
#configure_plotly_browser_state()
st='NV'
state_data = df[df.state == st]

# Elbow method to determine the number of K in Kmeans Clustering
coords = state_data[['longitude','latitude']]

distortions = []
K = range(1,25)
for k in K:
    kmeansModel = KMeans(n_clusters=k)
    kmeansModel = kmeansModel.fit(coords)
    distortions.append(kmeansModel.inertia_)

from sklearn.metrics import silhouette_score

sil = []
kmax = 50

# dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
for k in range(2, kmax+1):
  kmeans = KMeans(n_clusters = k).fit(coords)
  labels = kmeans.labels_
  sil.append(silhouette_score(coords, labels, metric = 'euclidean'))
  
kmeans = KMeans(n_clusters=5, init='k-means++')
kmeans.fit(coords)
y = kmeans.labels_
state_data['cluster'] = kmeans.predict(state_data[['longitude','latitude']])

top_stores = state_data.sort_values(by=['review_count', 'stars'], ascending=False)

def recommend_stores(df, longitude, latitude):
    # Predict the cluster for longitude and latitude provided
    cluster = kmeans.predict(np.array([longitude,latitude]).reshape(1,-1))[0]
    print(cluster)
   
    # Get the best restaurant in this cluster
    return  df[df['cluster']==cluster].iloc[0:5][['furniture_shop_no', 'latitude','longitude']]
   





'apt' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.


  Cloning git://github.com/geopandas/geopandas.git to c:\users\dell\appdata\local\temp\pip-req-build-0776zbi7


  Running command git clone -q git://github.com/geopandas/geopandas.git 'C:\Users\Dell\AppData\Local\Temp\pip-req-build-0776zbi7'
  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git clone -q git://github.com/geopandas/geopandas.git 'C:\Users\Dell\AppData\Local\Temp\pip-req-build-0776zbi7'
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


<ipython-input-1-9a3e56f6c0f5>:64: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [25]:
#Frontend

from tkinter import *
from tkinter import ttk
import sqlite3

window=Tk()
window.title("Timbernet")
label = Label(window,text="Timbernet",font=('arial',20,'bold'),bg='black',fg='white')
label.pack(side=TOP,fill=X)
label = Label(window,text="",font=('arial',15,'bold'),bg='black',fg='white')
label.pack(side=BOTTOM,fill=X)

def create():
    conn=sqlite3.connect('database.db')
    c=conn.cursor()
    c.execute("CREATE TABLE IF NOT EXISTS Maps(id integer primary key autoincrement,latitude TEXT,longitude TEXT,sqltime TIMESTAMP DEFAULT CURRENT)")
    conn.commit()
    conn.close()
create()

label = Label(window,text="Latitude",font=('arial',13,'bold'))
label.place(x=30,y=60)

name_entry=StringVar()
name_entry=ttk.Entry(window, textvariable=name_entry)
name_entry.place(x=130,y=60)
name_entry.focus()

label = Label(window,text="Longitude",font=('arial',13,'bold'))
label.place(x=30,y=100)

name_entry2=StringVar()
name_entry2=ttk.Entry(window, textvariable=name_entry2)
name_entry2.place(x=130,y=100)

name_entry3=StringVar()
label = Label(window,text="Address",font=('arial',13,'bold'))
label.place(x=30,y=140)

name_entry3=ttk.Entry(window, textvariable=name_entry3)
name_entry3.place(x=130,y=140)

label = Label(window,text="ZIP Code",font=('arial',13,'bold'))
label.place(x=30,y=180)

name_entry4=StringVar()
name_entry4=ttk.Entry(window, textvariable=name_entry4)
name_entry4.place(x=130,y=180)


def savedata():
    conn=sqlite3.connect('database.db')
    c=conn.cursor()
    c.execute('INSERT INTO Maps(latitude,longitude)VALUES (?,?)',(name_entry.get(),name_entry2.get()))
    conn.commit()
    conn.close()

btn=ttk.Button(window,text='Track Nearby Stores',command=savedata)
btn.place(x=130,y=220,width=125,height=30)



import pandas as pd
db = sqlite3.connect("database.db")
data = pd.read_sql_query('Select * from Maps',db)
data = data.tail(1)
#36.101732
user_latitude = float(data['latitude'])
#-115.189169
user_longitude = float(data['longitude'])

test_coordinates = {
    'latitude' : [user_latitude],
    'longitude' : [user_longitude],
}

user = pd.DataFrame(test_coordinates)

fig = px.scatter_mapbox(recommend_stores(top_stores, user_longitude, user_latitude), lat="latitude", lon="longitude",  
                   zoom=12, width=1200, height=800, hover_data= ['furniture_shop_no', 'latitude', 'longitude'])
fig = fig.add_scattermapbox(lat=user['latitude'], lon= user['longitude']).update_traces(dict(mode='markers', marker = dict(size = 15)))


window.geometry('500x500')
#window.resizable(False,False)
window.mainloop()




ValueError: could not convert string to float: ''

In [23]:
fig